In [3]:
import time
from concurrent.futures import ThreadPoolExecutor, thread
import concurrent.futures
import pandas as pd
from datetime import datetime, timedelta
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import re
import numpy as np
import psycopg2
import numpy as np
from scipy.stats import norm
import scipy.stats as stats
from sqlalchemy import create_engine
import zipfile
from functools import wraps
from typing import Any

In [4]:
connection = psycopg2.connect(host="10.5.1.20", database="marketdata", user="elliott", password="scalp", port='5432')
connection2 = psycopg2.connect(host="10.7.8.59", database="theoretical", user="scalp", password="QAtr@de442", port='5433')

m_root = 'RBadmin'
m_password = '$Calp123'
m_host = '10.5.1.32'
m_db = 'rbandits2'
uri = f"mysql+mysqldb://{m_root}:{m_password}@{m_host}/{m_db}"
mydb = create_engine(uri, connect_args={'ssl_mode': 'DISABLED'})
mapping = {0: "Vertical",5: "Straddle", 6: "Strangle"}

In [5]:
def timing_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        print(f"Elapsed time for {func.__name__}: {end_time - start_time} seconds")
        return result
    return wrapper

In [6]:
def convert_to_new_format(option):
    if option is None:
        return ''  # Return an empty string for NaN values

    parts = option.split('_')  # Split the spread into individual options
    new_parts = []
    for part in parts:
        # Identify the beginning of the date substring
        date_start = None
        for i in range(len(part) - 5):  # Subtract 5 to avoid running off the end of the string
            if part[i:i+2].isdigit() and part[i+2:i+4].isdigit() and part[i+4:i+6].isdigit():
                date_start = i
                break
        # If a date substring couldn't be found, treat the part as a non-option
        if date_start is None:
            new_parts.append(part)
            continue
        # Extract underlying, date, call/put, strike, and suffix
        underlying = part[:date_start]
        date = '20' + part[date_start:date_start+6]  # Convert YY to YYYY
        cp = part[date_start+6]
        strike_start = date_start + 7
        strike_end = strike_start
        for char in part[strike_start:]:
            if char.isdigit() or char == '.':
                strike_end += 1
            else:
                break
        strike = part[strike_start:strike_end]
        # Append decimal and trailing zeros if necessary
        if '.' not in strike:
            strike += '.00'
        elif len(strike.split('.')[1]) == 1:
            strike += '0'
        suffix = part[strike_end:]
        new_part = underlying + date + cp + strike + suffix
        new_parts.append(new_part)
    return '_'.join(new_parts) 

In [7]:
def load_and_preprocess_main_df(date):
    file_path = f'/home/elliott/Development/files/spreads-{date}0915.csv'
    
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        return None

    main_array = np.genfromtxt(file_path, delimiter=',', skip_header=1, dtype=None, encoding='utf-8')
    
    print("Shape of main_array:", main_array.shape)

    if len(main_array.shape) == 1:
        print("Array is 1-dimensional. Something went wrong.")
        return None
        
    # Keep only the columns we're interested in
    main_array = main_array[:, [0, 2, 3, 4, 5]]
    
    # Filter rows where both 'BBOBid' and 'BBOAsk' are zero
    main_array = main_array[(main_array[:, 1] != '0') | (main_array[:, 2] != '0')]

    # Add a new column for formatted symbols
    formatted_symbols = np.array([convert_to_new_format(symbol) for symbol in main_array[:, 0]], dtype=object)[:, np.newaxis]
    main_array = np.hstack([main_array, formatted_symbols])

    return main_array

In [9]:
main_df = load_and_preprocess_main_df('20230914')

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed